In [1]:
import os
from collections import defaultdict, Counter

from utils import load_embeddings, load_projects_new, load_stopwords

In [2]:
projects, imapping = load_projects_new('.', 'java-projects .final.csv')
labels = {k:v for k,v in zip(projects['names'], projects['labels_id'])}

In [3]:
path = "resources/java/stopwords.txt"
stopwords = load_stopwords(path)
path = "resources/en/stopwords.txt"
stopwords.update(load_stopwords(path))

In [4]:

terms_path = '../data/embeddings/terms-count/'
mapping = {v:k for k,v in imapping.items()}

In [5]:
from textblob import  Word

category_terms_count = defaultdict(lambda: Counter())
category_terms_occ = defaultdict(lambda: Counter())
text = []
lab = []
for project in labels:
    category = mapping[labels[project]]
    if category in ['Miscellaneous', '']:
        print('Skipped', project)
        continue
    try:
        terms_count = load_embeddings(os.path.join(terms_path, f"{project}.vec"))
        terms = []
        for x, y in terms_count.items():
            lemma = Word(x).lemmatize()
            if lemma not in stopwords and len(x) > 1 and x not in stopwords:
                tokens = [lemma] * int(y[0])
                terms.extend(tokens)
        #terms = [[Word(x).lemmatize()] * int(y[0]) for x, y in terms_count.items() if Word(x).lemmatize() not in stopwords and len(x) > 1]
        text.append(" ".join(terms))
        lab.append(category)
    except Exception as e:
        print(e)

[Errno 2] No such file or directory: '../data/embeddings/terms-count/asm.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/infinispan.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/infer.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/source.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/org.aspectj.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/JGroups.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/checker-framework.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/libgdx.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/eclipse.platform.swt.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/reflectasm.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/flink.vec'
[Errno 2] No such file or directory: '../data/embeddings/terms-count/Smack.

In [6]:
import pandas

df = pandas.DataFrame({'text': text, 'label': lab})
df['label'] = df['label'].fillna('NA')
df = df[~df['label'].isin(['NA', 'Miscellaneous'])]

y = df['label'].to_frame()
X = df['text']



from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(max_features=1000)
vectorizer.fit(X)
X_mat = vectorizer.transform(X)

In [7]:
X_mat = X_mat.todense()

In [8]:
len(vectorizer.get_feature_names())

1000

In [9]:
type(X_mat)

numpy.matrix

In [10]:
out = pandas.DataFrame(X_mat, columns=vectorizer.get_feature_names())

In [11]:
out.head()

,absolute,accept,access,accessor,accessors,account,ack,action,active,actor,...,wrap,wrapper,write,writer,xlink,xml,year,yield,zero,zone
0,0.000054,0.004101,0.000087,0.000075,0.000000,0.770056,0.0,0.000581,0.0,0.0,...,0.000000,0.006776,0.019786,0.016711,0.0,0.000000,0.004608,0.0,0.00000,0.000252
1,0.000000,0.000000,0.005596,0.004512,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000265,0.001247,0.005690,0.000000,0.0,0.085151,0.000000,0.0,0.00000,0.001892
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000
3,0.000000,0.001930,0.000487,0.172625,0.039102,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.027479,0.020356,0.020407,0.0,0.020996,0.000000,0.0,0.01673,0.003323
4,0.000000,0.000280,0.032079,0.083202,0.068412,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.018271,0.000477,0.075876,0.0,0.008466,0.000000,0.0,0.00000,0.001461


In [12]:
'class' in vectorizer.get_feature_names()

False

In [13]:
out['y'] = df['label']

In [14]:
out.head()

,absolute,accept,access,accessor,accessors,account,ack,action,active,actor,...,wrapper,write,writer,xlink,xml,year,yield,zero,zone,y
0,0.000054,0.004101,0.000087,0.000075,0.000000,0.770056,0.0,0.000581,0.0,0.0,...,0.006776,0.019786,0.016711,0.0,0.000000,0.004608,0.0,0.00000,0.000252,Introspection
1,0.000000,0.000000,0.005596,0.004512,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.001247,0.005690,0.000000,0.0,0.085151,0.000000,0.0,0.00000,0.001892,Introspection
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,Introspection
3,0.000000,0.001930,0.000487,0.172625,0.039102,0.000000,0.0,0.000000,0.0,0.0,...,0.027479,0.020356,0.020407,0.0,0.020996,0.000000,0.0,0.01673,0.003323,Introspection
4,0.000000,0.000280,0.032079,0.083202,0.068412,0.000000,0.0,0.000000,0.0,0.0,...,0.018271,0.000477,0.075876,0.0,0.008466,0.000000,0.0,0.00000,0.001461,Introspection


In [15]:
out.to_csv('our_tfidf_1000_wo_miscellaneous.csv', index=False)